# EMA Cross Chart

This notebook demonstrates an EMA (Exponential Moving Average) crossover strategy using vectorbt for backtesting.

In [18]:
import vectorbt as vbt
import yfinance as yf
import pandas as pd

## Set up parameters

In [19]:
TICKER = 'SPY'
EMA_FAST = 8
EMA_SLOW = 21

## Download historical data

In [20]:
data = yf.download(TICKER, interval='1d', period='max')
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,24.608624,1003200
1993-02-01,43.96875,44.25000,43.96875,44.25000,24.783646,480500
1993-02-02,44.21875,44.37500,44.12500,44.34375,24.836164,201300
1993-02-03,44.40625,44.84375,44.37500,44.81250,25.098709,529400
1993-02-04,44.96875,45.09375,44.46875,45.00000,25.203711,531500


## Calculate EMAs

In [21]:
data['EMA_short'] = data['Close'].ewm(span=EMA_FAST, adjust=False).mean()
data['EMA_long'] = data['Close'].ewm(span=EMA_SLOW, adjust=False).mean()
data.tail()

,Open,High,Low,Close,Adj Close,Volume,EMA_short,EMA_long
Date,,,,,,,,
2024-10-10,575.770020,577.580017,574.489990,576.130005,576.130005,44138100,572.779711,568.239457
2024-10-11,576.049988,580.330017,575.909973,579.580017,579.580017,42268000,574.290890,569.270417
2024-10-14,581.219971,585.270020,580.729980,584.320007,584.320007,36217200,576.519583,570.638562
2024-10-15,584.590027,584.900024,578.539978,579.780029,579.780029,54111400,577.244127,571.469604
2024-10-16,579.780029,582.820007,578.960022,582.299988,582.299988,30606126,578.367651,572.454184


## Generate signals

In [22]:
entries = data['EMA_short'] > data['EMA_long']
exits = data['EMA_short'] < data['EMA_long']

## Backtest using vectorbt

In [23]:
pf = vbt.Portfolio.from_signals(data['Close'], entries, exits)

## Analyze and plot the results

In [24]:
print(pf.stats())

Start                         1993-01-29 00:00:00
End                           2024-10-16 00:00:00
Period                                       7986
Start Value                                 100.0
End Value                              336.048033
Total Return [%]                       236.048033
Benchmark Return [%]                   1225.29158
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        39.347818
Max Drawdown Duration                      3854.0
Total Trades                                  185
Total Closed Trades                           184
Total Open Trades                               1
Open Trade PnL                          13.394576
Win Rate [%]                            39.130435
Best Trade [%]                          29.925525
Worst Trade [%]                         -7.068119
Avg Winning Trade [%]                    5.103268
Avg Losing Trade [%]                    -2.049866


c:\Projects\trading\.venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Projects\trading\.venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Projects\trading\.venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Projects\trading\.venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



In [25]:
pf.plot().show()